In [1]:
from pymongo import MongoClient
import pandas as pd
from datetime import datetime

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
client = MongoClient('mongodb://localhost:27017/')

In [3]:
db = client.music_db

In [4]:
albums_collection = db.albums
artists_collection = db.artists
playlists_collection = db.playlists
songs_collection = db.songs
users_collection = db.users

In [5]:
results = songs_collection.find({})

results_list = list(results)

df = pd.DataFrame(results_list)
df.head(10)

,_id,Song_ID,Title,Artist_ID,Album_ID,Playlist_ID,Genre,Rating,Duration
0,66564091e7d2ff466e50302f,1,Happier,1.0,1.0,1.0,Pop,7,4:16
1,66564091e7d2ff466e503030,2,Enough for You,1.0,1.0,4.0,Pop,6,2:47
2,66564091e7d2ff466e503031,3,Deja Vu,1.0,1.0,7.0,Pop,10,4:01
3,66564091e7d2ff466e503032,4,Brutal,1.0,1.0,10.0,Pop,5,3:18
4,66564091e7d2ff466e503033,5,Kiss Me More,9.0,7.0,5.0,Pop,8,3:12
5,66564091e7d2ff466e503034,6,American Idiot,3.0,8.0,8.0,Rock,7,3:24
6,66564091e7d2ff466e503035,7,Boulevard of Broken Dreams,3.0,8.0,10.0,Rock,8,4:45
7,66564091e7d2ff466e503036,8,Homecoming,3.0,8.0,9.0,Rock,6,8:46
8,66564091e7d2ff466e503037,9,Golden,8.0,2.0,1.0,Pop,7,3:50
9,66564091e7d2ff466e503038,10,Adore You,8.0,2.0,5.0,Pop,7,4:25


,_id,Title,Genre,Rating,Duration,Album_Title,Album_Release_Date,Artist_Name,Artist_Country
0,6,American Idiot,Rock,7,3:24,American Idiot,2004-07-25 00:00:00,Green Day,USA
1,37,Sign of the Times,Rock,9,5:40,Fine Line,2019-06-03 00:00:00,Harry Styles,UK
2,7,Boulevard of Broken Dreams,Rock,8,4:45,American Idiot,2004-07-25 00:00:00,Green Day,USA
3,8,Homecoming,Rock,6,8:46,American Idiot,2004-07-25 00:00:00,Green Day,USA
4,21,Yellow,Rock,9,4:29,Parachutes,0200-07-10 00:00:00,Coldplay,USA
5,24,Trouble,Rock,9,4:30,Parachutes,0200-07-10 00:00:00,Coldplay,USA
6,22,Shiver,Rock,8,4:59,Parachutes,0200-07-10 00:00:00,Coldplay,USA
7,23,Sparks,Rock,8,3:47,Parachutes,0200-07-10 00:00:00,Coldplay,USA


In [7]:
pipeline = [
    {
        "$sort": {
            "Rating": -1
        }
    },
    {
        "$group": {
            "_id": "$Artist_ID",
            "Top_Rated_Song": {
                "$first": "$$ROOT"
            }
        }
    },
    {
        "$lookup": {
            "from": "artists",
            "localField": "_id",
            "foreignField": "Artist_ID",
            "as": "artist_info"
        }
    },
    {
        "$unwind": "$artist_info"
    },
    {
        "$project": {
            "_id": 0,
            "Artist_Name": "$artist_info.Artist_Name",
            "Title": "$Top_Rated_Song.Title",
            "Rating": "$Top_Rated_Song.Rating",
            "Genre": "$Top_Rated_Song.Genre",
            "Duration": "$Top_Rated_Song.Duration"
        }
    }
]

results = songs_collection.aggregate(pipeline)
df = pd.DataFrame(list(results))
df.head()


,Artist_Name,Title,Rating,Genre,Duration
0,Connor Price,Spinnin,9,Pop,1:50
1,Connor Price,Spinnin,9,Pop,1:50
2,Connor Price,Spinnin,9,Pop,1:50
3,Connor Price,Spinnin,9,Pop,1:50
4,Connor Price,Spinnin,9,Pop,1:50


In [8]:
pipeline = [
    {
        "$lookup": {
            "from": "users",
            "localField": "Creator_ID",
            "foreignField": "Creator_ID",
            "as": "user_info"
        }
    },
    {
        "$unwind": "$user_info"
    },
    {
        "$match": {
            "user_info.Country": "USA"  
        }
    },
    {
        "$project": {
            "_id": 0,
            "Playlist_name": 1,
            "Description": 1,
            "Popularity": 1,
            "Username": "$user_info.Username",
            "User_Country": "$user_info.Country"
        }
    }
]

results = playlists_collection.aggregate(pipeline)
df = pd.DataFrame(list(results))
df.head()


,Playlist_name,Description,Popularity,Username,User_Country
0,Driving Time,Songs to drive to.,1000,Blink21,USA
1,Driving Time,Songs to drive to.,1000,Blink21,USA
2,Driving Time,Songs to drive to.,1000,Blink21,USA
3,Driving Time,Songs to drive to.,1000,Blink21,USA
4,Driving Time,Songs to drive to.,1000,Blink21,USA


In [9]:
pipeline = [
    {
        "$match": {
            "Rating": { "$gt": 7.0 }  
        }
    },
    {
        "$lookup": {
            "from": "albums",
            "localField": "Album_ID",
            "foreignField": "Album_ID",
            "as": "album_info"
        }
    },
    {
        "$lookup": {
            "from": "artists",
            "localField": "Artist_ID",
            "foreignField": "Artist_ID",
            "as": "artist_info"
        }
    },
    {
        "$unwind": "$album_info"
    },
    {
        "$unwind": "$artist_info"
    },
    {
        "$project": {
            "_id": 0,
            "Song_ID": 1,
            "Title": 1,
            "Genre": 1,
            "Rating": 1,
            "Duration": 1,
            "Album_Title": "$album_info.Album_title",
            "Album_Release_Date": "$album_info.Album_release_date",
            "Artist_Name": "$artist_info.Artist_Name",
            "Artist_Country": "$artist_info.Artist_Country"
        }
    },
    {
        "$group": {
            "_id": "$Song_ID",
            "Title": { "$first": "$Title" },
            "Genre": { "$first": "$Genre" },
            "Rating": { "$first": "$Rating" },
            "Duration": { "$first": "$Duration" },
            "Album_Title": { "$first": "$Album_Title" },
            "Album_Release_Date": { "$first": "$Album_Release_Date" },
            "Artist_Name": { "$first": "$Artist_Name" },
            "Artist_Country": { "$first": "$Artist_Country" }
        }
    }
]

results = songs_collection.aggregate(pipeline)
df = pd.DataFrame(list(results))
df.head(10)

,_id,Title,Genre,Rating,Duration,Album_Title,Album_Release_Date,Artist_Name,Artist_Country
0,26,All I Want,Indie,8,5:05,In A Perfect World,2013-06-17 00:00:00,Kodaline,USA
1,40,Golden (Live),Pop,8,3:28,Fine Line,2019-06-03 00:00:00,Harry Styles,UK
2,55,AOK,Pop,8,2:53,TV,2021-05-20 00:00:00,Tai Verdes,USA
3,37,Sign of the Times,Rock,9,5:40,Fine Line,2019-06-03 00:00:00,Harry Styles,UK
4,38,Adore You (Live),Pop,8,3:27,Fine Line,2019-06-03 00:00:00,Harry Styles,UK
5,49,Take Me to Church,Indie,10,4:01,Unheard,2024-03-22 00:00:00,Hozier,Ireland
6,27,High Hopes,Indie,8,3:51,In A Perfect World,2013-06-17 00:00:00,Kodaline,USA
7,23,Sparks,Rock,8,3:47,Parachutes,0200-07-10 00:00:00,Coldplay,USA
8,35,Geeks,Pop,8,2:55,Hardwired Mixtape,2018-11-16 00:00:00,Hailey Knox,USA
9,3,Deja Vu,Pop,10,4:01,Sour,2019-07-20 00:00:00,Olivia Rodrigo,USA


In [10]:
client = MongoClient('localhost', 27017)
db = client.music_database  # Replace 'music_database' with your actual database name

# Define the 'albums' collection
albums = db.albums

# Query to find albums with release dates after 2018
query = {"Album_release_date": {"$gt": datetime(2018, 12, 31)}}
filtered_albums = albums.find(query)

# Print each album that matches the query
for album in filtered_albums:
    print(album)

In [17]:
pipeline = [
    {
        "$lookup": {
            "from": "albums",
            "localField": "Album_ID",
            "foreignField": "Album_ID",
            "as": "album_info"
        }
    },
    {
        "$unwind": "$album_info"
    },
    {
        "$match": {
            "album_info.Album_release_date": { "$gt": "2018-12-31" }  
        }
    },
    {
        "$lookup": {
            "from": "artists",
            "localField": "Artist_ID",
            "foreignField": "Artist_ID",
            "as": "artist_info"
        }
    },
    {
        "$unwind": "$artist_info"
    },
    {
        "$project": {
            "_id": 0,
            "Song_ID": 1,
            "Title": 1,
            "Genre": 1,
            "Rating": 1,
            "Duration": 1,
            "Album_Title": "$album_info.Album_title",
            "Album_Release_Date": "$album_info.Album_release_date",
            "Artist_Name": "$artist_info.Artist_Name",
            "Artist_Country": "$artist_info.Artist_Country"
        }
    },
    {
        "$group": {
            "_id": "$Song_ID",
            "Title": { "$first": "$Title" },
            "Genre": { "$first": "$Genre" },
            "Rating": { "$first": "$Rating" },
            "Duration": { "$first": "$Duration" },
            "Album_Title": { "$first": "$Album_Title" },
            "Album_Release_Date": { "$first": "$Album_Release_Date" },
            "Artist_Name": { "$first": "$Artist_Name" },
            "Artist_Country": { "$first": "$Artist_Country" }
        }
    }
]

results = songs_collection.aggregate(pipeline)
df = pd.DataFrame(list(results))
df.head(50)

,_id,Title,Genre,Rating,Duration,Album_Title,Album_Release_Date,Artist_Name,Artist_Country
0,48,Too Sweet,Indie,7,4:11,Unheard,3/22/2024,Hozier,Ireland
1,37,Sign of the Times,Rock,9,5:40,Fine Line,6/3/2019,Harry Styles,UK
2,38,Adore You (Live),Pop,8,3:27,Fine Line,6/3/2019,Harry Styles,UK
3,49,Take Me to Church,Indie,10,4:01,Unheard,3/22/2024,Hozier,Ireland
4,4,Brutal,Pop,5,3:18,Sour,7/20/2019,Olivia Rodrigo,USA
5,40,Golden (Live),Pop,8,3:28,Fine Line,6/3/2019,Harry Styles,UK
6,50,Work Song,Indie,6,3:49,Unheard,3/22/2024,Hozier,Ireland
7,25,Don't Panic,Alternative Rock,8,2:17,Parachutes,7/10/2000,Coldplay,USA
8,9,Golden,Pop,7,3:50,Fine Line,6/3/2019,Harry Styles,UK
9,29,All Comes Down,Indie,7,4:01,In A Perfect World,6/17/2013,Kodaline,USA
